<a href="https://colab.research.google.com/github/sinash838/Keras_LSTM_Text_Generator/blob/master/Copy_of_Copy_of_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text generation with an RNN

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/text/tutorials/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial demonstrates how to generate text using a character-based RNN. You will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: *Runtime > Change runtime type > Hardware accelerator > GPU*.

This tutorial includes runnable code implemented using [tf.keras](https://www.tensorflow.org/guide/keras/sequential_model) and [eager execution](https://www.tensorflow.org/guide/eager). The following is the sample output when the model in this tutorial trained for 30 epochs, and started with the prompt "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving hafez.txt to hafez.txt


In [ ]:
example1 = "/content/hafez.txt"
file1 = open(example1, "r")

In [ ]:
#path_to_file = tf.keras.utils.get_file('hafez.txt', '/content/hafez.txt')
text = open(example1, 'rb').read().decode(encoding='utf-8')
print(f'Length of text: {len(text)} characters')

Length of text: 296713 characters


### Read the data

First, look in the text:

In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

غزل   ۱

الا يا ايها الساقی ادر کاسا و ناولها
که عشق آسان نمود اول ولی افتاد مشکل‌ها

به بوی نافه‌ای کاخر صبا زان طره بگشايد
ز تاب جعد مشکينش چه خون افتاد در دل‌ها

مرا در منزل جانان چه امن عيش چون هر دم
جرس فرياد می‌دارد که بربنديد محمل‌ها


In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

49 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

The `preprocessing.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [ ]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `preprocessing.StringLookup` layer:

In [ ]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts form tokens to character IDs:

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `preprocessing.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `preprocessing.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [ ]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]', b'[UNK]'], [b'[UNK]', b'[UNK]', b'[UNK]']]>

You can `tf.strings.reduce_join` to join the characters back into strings. 

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'[UNK][UNK][UNK][UNK][UNK][UNK][UNK]', b'[UNK][UNK][UNK]'],
      dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `seq_length` characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of `seq_length+1`. For example, say `seq_length` is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(296713,), dtype=int64, numpy=array([24, 16, 27, ..., 28,  2,  1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

غ
ز
ل
 
 
 
۱






In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'\xd8\xba' b'\xd8\xb2' b'\xd9\x84' b' ' b' ' b' ' b'\xdb\xb1' b'\r'
 b'\n' b'\r' b'\n' b'\xd8\xa7' b'\xd9\x84' b'\xd8\xa7' b' ' b'\xd9\x8a'
 b'\xd8\xa7' b' ' b'\xd8\xa7' b'\xd9\x8a' b'\xd9\x87' b'\xd8\xa7' b' '
 b'\xd8\xa7' b'\xd9\x84' b'\xd8\xb3' b'\xd8\xa7' b'\xd9\x82' b'\xdb\x8c'
 b' ' b'\xd8\xa7' b'\xd8\xaf' b'\xd8\xb1' b' ' b'\xda\xa9' b'\xd8\xa7'
 b'\xd8\xb3' b'\xd8\xa7' b' ' b'\xd9\x88' b' ' b'\xd9\x86' b'\xd8\xa7'
 b'\xd9\x88' b'\xd9\x84' b'\xd9\x87' b'\xd8\xa7' b'\r' b'\n' b'\xda\xa9'
 b'\xd9\x87' b' ' b'\xd8\xb9' b'\xd8\xb4' b'\xd9\x82' b' ' b'\xd8\xa2'
 b'\xd8\xb3' b'\xd8\xa7' b'\xd9\x86' b' ' b'\xd9\x86' b'\xd9\x85'
 b'\xd9\x88' b'\xd8\xaf' b' ' b'\xd8\xa7' b'\xd9\x88' b'\xd9\x84' b' '
 b'\xd9\x88' b'\xd9\x84' b'\xdb\x8c' b' ' b'\xd8\xa7' b'\xd9\x81'
 b'\xd8\xaa' b'\xd8\xa7' b'\xd8\xaf' b' ' b'\xd9\x85' b'\xd8\xb4'
 b'\xda\xa9' b'\xd9\x84' b'\xe2\x80\x8c' b'\xd9\x87' b'\xd8\xa7' b'\r'
 b'\n' b'\r' b'\n' b'\xd8\xa8' b'\xd9\x87' b' ' b'\xd8\xa8' b'\xd9\x88'
 b'\x

It's easier to see what this is doing if you join the tokens back into strings:

In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'\xd8\xba\xd8\xb2\xd9\x84   \xdb\xb1\r\n\r\n\xd8\xa7\xd9\x84\xd8\xa7 \xd9\x8a\xd8\xa7 \xd8\xa7\xd9\x8a\xd9\x87\xd8\xa7 \xd8\xa7\xd9\x84\xd8\xb3\xd8\xa7\xd9\x82\xdb\x8c \xd8\xa7\xd8\xaf\xd8\xb1 \xda\xa9\xd8\xa7\xd8\xb3\xd8\xa7 \xd9\x88 \xd9\x86\xd8\xa7\xd9\x88\xd9\x84\xd9\x87\xd8\xa7\r\n\xda\xa9\xd9\x87 \xd8\xb9\xd8\xb4\xd9\x82 \xd8\xa2\xd8\xb3\xd8\xa7\xd9\x86 \xd9\x86\xd9\x85\xd9\x88\xd8\xaf \xd8\xa7\xd9\x88\xd9\x84 \xd9\x88\xd9\x84\xdb\x8c \xd8\xa7\xd9\x81\xd8\xaa\xd8\xa7\xd8\xaf \xd9\x85\xd8\xb4\xda\xa9\xd9\x84\xe2\x80\x8c\xd9\x87\xd8\xa7\r\n\r\n\xd8\xa8\xd9\x87 \xd8\xa8\xd9\x88\xdb\x8c \xd9\x86\xd8\xa7\xd9\x81'
b'\xd9\x87\xe2\x80\x8c\xd8\xa7\xdb\x8c \xda\xa9\xd8\xa7\xd8\xae\xd8\xb1 \xd8\xb5\xd8\xa8\xd8\xa7 \xd8\xb2\xd8\xa7\xd9\x86 \xd8\xb7\xd8\xb1\xd9\x87 \xd8\xa8\xda\xaf\xd8\xb4\xd8\xa7\xd9\x8a\xd8\xaf\r\n\xd8\xb2 \xd8\xaa\xd8\xa7\xd8\xa8 \xd8\xac\xd8\xb9\xd8\xaf \xd9\x85\xd8\xb4\xda\xa9\xd9\x8a\xd9\x86\xd8\xb4 \xda\x86\xd9\x87 \xd8\xae\xd9\x88\xd9\x86 \xd8\xa7\xd9\x81\xd8\xaa\xd8

For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'\xd8\xba\xd8\xb2\xd9\x84   \xdb\xb1\r\n\r\n\xd8\xa7\xd9\x84\xd8\xa7 \xd9\x8a\xd8\xa7 \xd8\xa7\xd9\x8a\xd9\x87\xd8\xa7 \xd8\xa7\xd9\x84\xd8\xb3\xd8\xa7\xd9\x82\xdb\x8c \xd8\xa7\xd8\xaf\xd8\xb1 \xda\xa9\xd8\xa7\xd8\xb3\xd8\xa7 \xd9\x88 \xd9\x86\xd8\xa7\xd9\x88\xd9\x84\xd9\x87\xd8\xa7\r\n\xda\xa9\xd9\x87 \xd8\xb9\xd8\xb4\xd9\x82 \xd8\xa2\xd8\xb3\xd8\xa7\xd9\x86 \xd9\x86\xd9\x85\xd9\x88\xd8\xaf \xd8\xa7\xd9\x88\xd9\x84 \xd9\x88\xd9\x84\xdb\x8c \xd8\xa7\xd9\x81\xd8\xaa\xd8\xa7\xd8\xaf \xd9\x85\xd8\xb4\xda\xa9\xd9\x84\xe2\x80\x8c\xd9\x87\xd8\xa7\r\n\r\n\xd8\xa8\xd9\x87 \xd8\xa8\xd9\x88\xdb\x8c \xd9\x86\xd8\xa7'
Target: b'\xd8\xb2\xd9\x84   \xdb\xb1\r\n\r\n\xd8\xa7\xd9\x84\xd8\xa7 \xd9\x8a\xd8\xa7 \xd8\xa7\xd9\x8a\xd9\x87\xd8\xa7 \xd8\xa7\xd9\x84\xd8\xb3\xd8\xa7\xd9\x82\xdb\x8c \xd8\xa7\xd8\xaf\xd8\xb1 \xda\xa9\xd8\xa7\xd8\xb3\xd8\xa7 \xd9\x88 \xd9\x86\xd8\xa7\xd9\x88\xd9\x84\xd9\x87\xd8\xa7\r\n\xda\xa9\xd9\x87 \xd8\xb9\xd8\xb4\xd9\x82 \xd8\xa2\xd8\xb3\xd8\xa7\xd9\x86 \xd9\x86\xd9\x

### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). 

This model has three layers:

* `tf.keras.layers.Embedding`: The input layer. A trainable lookup table that will map each character-ID to a vector with `embedding_dim` dimensions;
* `tf.keras.layers.GRU`: A type of RNN with size `units=rnn_units` (You can also use an LSTM layer here.)
* `tf.keras.layers.Dense`: The output layer, with `vocab_size` outputs. It outputs one logit for each character in the vocabulary. These are the log-likelihood of each character according to the model.

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

For each character the model looks up the embedding, runs the GRU one timestep with the embedding as input, and applies the dense layer to generate logits predicting the log-likelihood of the next character:

![A drawing of the data passing through the model](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_training.png?raw=1)

Note: For training you could use a `keras.Sequential` model here. To  generate text later you'll need to manage the RNN's internal state. It's simpler to include the state input and output options upfront, than it is to rearrange the model architecture later. For more details see the [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 50) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  12800     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  51250     
Total params: 4,002,354
Trainable params: 4,002,354
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [ ]:
sampled_indices

array([12, 43, 11, 31, 35, 46,  1, 26, 10, 43, 31, 17, 13,  6,  7, 15, 18,
        0, 48, 23, 41, 42, 41, 23, 33, 16, 48, 29, 21, 18, 22, 18, 49, 32,
       20, 22, 40, 22, 24, 30, 18,  3, 20, 35, 45, 33,  7,  8, 43, 19, 25,
       19,  7, 12, 31, 25, 37, 48, 43, 26, 39, 10, 31,  0, 42, 44, 25, 46,
       28, 18, 25, 23, 13, 34, 41, 16, 14, 29, 35, 20,  0, 22, 38,  1, 29,
       20,  3, 43, 19, 41, 36, 44, 13,  7, 49, 26, 39, 13, 46, 22])

Decode these to see the text predicted by this untrained model:

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'\xd9\x8a\xd9\x86 \xd9\x85\xd8\xb9\xd9\x85\xd8\xa7 \xd8\xb4\xd8\xb1\xd8\xad \xd9\x88 \xd8\xa8\xd9\x8a\xd8\xa7\xd9\x86 \xd9\x86\xd8\xaf\xd8\xa7\xd8\xb1\xd8\xaf\r\n\r\n\xd8\xb3\xd8\xb1\xd9\x85\xd9\x86\xd8\xb2\xd9\x84 \xd9\x81\xd8\xb1\xd8\xa7\xd8\xba\xd8\xaa \xd9\x86\xd8\xaa\xd9\x88\xd8\xa7\xd9\x86 \xd8\xb2 \xd8\xaf\xd8\xb3\xd8\xaa \xd8\xaf\xd8\xa7\xd8\xaf\xd9\x86\r\n\xd8\xa7\xdb\x8c \xd8\xb3\xd8\xa7\xd8\xb1\xd9\x88\xd8\xa7\xd9\x86 \xd9\x81\xd8\xb1\xd9\x88\xda\xa9\xd8\xb4 \xda\xa9\xd8\xa7\xd9\x8a\xd9\x86 \xd8\xb1\xd9\x87 \xda\xa9\xd8\xb1\xd8\xa7\xd9\x86 \xd9\x86\xd8\xaf\xd8\xa7\xd8\xb1\xd8\xaf\r\n\r\n\xda\x86\xd9\x86\xda\xaf'

Next Char Predictions:
 b'\xd8\xae\xdb\xb4\xd8\xad\xd9\x88\xda\x98\xdb\xb7\n\xd9\x82\xd8\xac\xdb\xb4\xd9\x88\xd8\xb3\xd8\xaf\xd8\xa7\xd8\xa8\xd8\xb1\xd8\xb4[UNK]\xdb\xb9\xd8\xb9\xdb\xb2\xdb\xb3\xdb\xb2\xd8\xb9\xd9\xbe\xd8\xb2\xdb\xb9\xd9\x86\xd8\xb7\xd8\xb4\xd8\xb8\xd8\xb4\xe2\x80\x8c\xd9\x8a\xd8\xb6\xd8\xb8\xdb\xb1\xd8\xb8\xd8\xba\xd9\x87\xd8\xb4 \xd8\xb6\

## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 50)  # (batch_size, sequence_length, vocab_size)
Mean loss:         3.911028


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [ ]:
tf.exp(mean_loss).numpy()

49.950268

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [ ]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [ ]:
EPOCHS = 20

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
45/45 [==============================] - 9s 136ms/step - loss: 3.4025
Epoch 2/20
45/45 [==============================] - 6s 134ms/step - loss: 2.5457
Epoch 3/20
45/45 [==============================] - 6s 134ms/step - loss: 2.3633
Epoch 4/20
45/45 [==============================] - 6s 134ms/step - loss: 2.2630
Epoch 5/20
45/45 [==============================] - 6s 135ms/step - loss: 2.1734
Epoch 6/20
45/45 [==============================] - 7s 135ms/step - loss: 2.1026
Epoch 7/20
45/45 [==============================] - 7s 134ms/step - loss: 2.0481
Epoch 8/20
45/45 [==============================] - 6s 134ms/step - loss: 1.9955
Epoch 9/20
45/45 [==============================] - 7s 135ms/step - loss: 1.9483
Epoch 10/20
45/45 [==============================] - 6s 133ms/step - loss: 1.9021
Epoch 11/20
45/45 [==============================] - 6s 134ms/step - loss: 1.8529
Epoch 12/20
45/45 [==============================] - 6s 135ms/step - loss: 1.8074
Epoch 13/20
45/45 [======

## Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

![To generate text the model's output is fed back to the input](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_sampling.png?raw=1)

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['از من'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

از من اوايای چو مندگانه حياتم کز آن دل‌هاست که می‌گويم
با چشم بنياد که اين قير و جمش دل بی بخت

چو لنش آيد هم از نظر فلن بی‌آفتر زنم
چند که حوم نکرد که ماه می‌کنيم

تا از ده مولا و نو عرش کن و دهنان باش
عارف بيار باده اهل رضا بازد


غزل    ۲۴۴

دل بگويم احسباک در می مغنوا
که غمزه گر رايت رفت و کشانم چو شمع

اين نقيل که مويی و معلوم کن به فرياد
به دست سر سودای توان گذر تذار

عقوق کلک هيچ انديشش خداير مکن
حافظ همت لحيقه رضو با می‌برد
نفس حافظ ای شود که با سوسن بگد که

به اين دردی کافتی که خمن بخش روی
ز کی ببر آن است که اقفام غلام باخت

چو پند نيست معذوری و ادب نامه چون
اگر نه که دود آن اول که ز سربوزر و می‌کرد

کجاست هجرا رب که نهله را به مور
به ياقوش نه دلداری اندر کوی زلف

نگشت مينم دل لايقی لانع هر کيست
همچو لکل دوست به وصل تو خاک که تو بکند

بکشده زندم که به درت ورده کهی
هر آن چين کز اين کمينگ عشق مسمايا بلنگار
کزار دست شمشيد که شاهيا و دست

لشمه‌ای کن حقر و فلک به هر شو وجود
ور بلبلی به که در اين کاريست

نکته آن که چه شاطی که جفايت بازم
از وانعه  

__________________________________

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b'ROMEO:\xd9\x88\xd8\xb4\xd9\x85\r\n\r\n\r\n\xd8\xba\xd8\xb2\xd9\x84    \xdb\xb3\xdb\xb1\xdb\xb0\r\n\r\n\xd8\xa7\xdb\x8c \xd8\xb9\xd8\xa7\xd8\xb4\xd9\x82 \xda\xa9\xd9\x87 \xd8\xac\xd8\xa7\xd9\x85 \xd9\x85\xd8\xb4\xd8\xaa\xd9\x82\xd8\xa8\xd8\xa7 \xd8\xaf\xd8\xa7\xd8\xb4\xd8\xaa\r\n\xda\xa9\xd9\x87 \xd9\x82\xd8\xa7\xd9\x84\xd8\xa8 \xd9\x86\xd8\xa8\xd9\x88\xd8\xaf \xd8\xa8\xd8\xa8\xd8\xb1 \xd8\xb2 \xd9\x85\xd8\xb1 \xd8\xa7\xd9\x86\xd8\xaa\xd8\xb8\xd8\xa7\xd8\xb1 \xda\xa9\xd8\xa7\xd8\xb1\r\n\r\n\xd8\xa8\xd8\xae\xd9\x88\xd8\xa7\xd9\x87 \xd8\xaf\xd8\xa7\xd8\xb1 \xd9\x84\xd9\x8a\xd9\x84\xdb\x8c \xd8\xa8\xd8\xac\xd8\xb2 \xd8\xac\xd8\xa7\xd9\x86 \xd8\xa8\xd8\xa7\xd8\xb1\xd8\xa7\xd9\x86 \xd8\xae\xd8\xa7\xd8\xb1\r\n\xda\xa9\xd8\xa7\xd9\x86 \xd8\xb4\xd9\x8a\xd9\x88\xd9\x87 \xda\xa9\xd9\x87 \xd8\xa8\xd8\xa7 \xd9\x85\xd9\x86 \xd8\xaf\xd9\x8a\xd8\xaf\xd9\x87 \xd9\x85\xd8\xaf\xd9\x85\xd8\xb9\xd9\x87 \xd8\xa2\xd8\xac\xd8\xb2\xdb\x8c\r\n\r\n\xda\xa9\xd8\xb3\xdb\x8c \xd8\xa8\xd8\xa7\xd8\xaf \

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


In [ ]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:له مجر جوسه که کافروخت
ای دل بويش در زلف يار گو دويش

در کار خداتی دام ره ايست
لاجرم به در ميخان


## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [ ]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
model.fit(dataset, epochs=1)

Or if you need more control, you can write your own complete custom training loop:

In [ ]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))